Load required packages 

In [2]:
import os 
import numpy as np 
import h5py
import scipy.io
import matlab_helpers as mh 
import tensorflow as tf 

Read the names of all mat-files 

In [3]:
root_matdata_dir = 'D:/Dropbox/Python/MLmodels/Datasets/mps_gp_sgram/'
out_ml_dir = root_matdata_dir + 'ml_data_CNN/'
if not os.path.exists(out_ml_dir):
    os.makedirs(out_ml_dir)

clean_in_data_dir = root_matdata_dir + 'level65_dBspl_clean_ds/'
valid_datadirs = ['Chut', 'HighWhistle', 'Rumble', 'Tchatter', 'Wheek', 'Whine']
calls2use = ['Chut', 'Rumble', 'Wheek', 'Whine']

all_files_nameonly= []
dsMPS_data_list= []

for cur_call_dir in valid_datadirs:
    cur_call_path = clean_in_data_dir + cur_call_dir + '/' 
    # print(cur_call_path)
    cur_dir_files = [f for f in os.listdir(cur_call_path) if os.path.isfile(os.path.join(cur_call_path, f))]
    all_files_nameonly = all_files_nameonly + cur_dir_files
    dsMPS_data_list = dsMPS_data_list + [cur_call_path + f for f in cur_dir_files]
    

print(f"--> all_files_nameonly: len lines= {len(all_files_nameonly)}, first line = {all_files_nameonly[0]}")
print(f"--> dsMPS_data_list: len lines= {len(dsMPS_data_list)}, first line = {dsMPS_data_list[0]}")

out_allfiles_txt_fname = out_ml_dir + 'all_dsMPS_data_list.txt'
with open(out_allfiles_txt_fname, 'w') as f:
    for line in dsMPS_data_list:
        f.write(f"{line}\n")

--> all_files_nameonly: len lines= 1605, first line = ds_Chut_2_Feb_07_2022_51861688_ms_101198_101787.mat
--> dsMPS_data_list: len lines= 1605, first line = D:/Dropbox/Python/MLmodels/Datasets/mps_gp_sgram/level65_dBspl_clean_ds/Chut/ds_Chut_2_Feb_07_2022_51861688_ms_101198_101787.mat


Read Mat files and then create .h5 files for training 

In [5]:
out_dsMPS_data_file = out_ml_dir + 'all_dsMPS_data.h5'
data_dsMPS_x = []
pre_search_str = '_ds/'
post_search_str = '/ds' 

for fName,iter in zip(dsMPS_data_list,np.arange(len(dsMPS_data_list))):
    data = mh.loadmat(fName)
    cur_data = data["mps_struct"]["mps_pow_dB"]
    data_dsMPS_x.append(cur_data) 
    if iter % 100 == 0:
        print(f"iter={iter} with file={fName}: size={cur_data.shape}")
    
    
data_label_name = [item[item.rfind(pre_search_str)+len(pre_search_str):item.rfind(post_search_str)] for item in dsMPS_data_list]
data_label_y = len(calls2use)*np.ones((len(data_dsMPS_x),1))
unq_vals, unq_counts = np.unique(data_label_y, return_counts=True)
print(dict(zip(unq_vals,unq_counts)))

for ind, cur_call in zip(np.arange(len(data_label_name)),data_label_name):
    if cur_call in calls2use: 
        data_label_y[ind,0] = calls2use.index(cur_call)

data_dsMPS_x = np.array(data_dsMPS_x)
data_label_y = np.array(data_label_y).astype(int)
print(f"data_dsMPS_x={type(data_dsMPS_x)}&{len(data_dsMPS_x)},data_label_y={type(data_label_y)}&{data_label_y.shape},")

if (not os.path.exists(out_dsMPS_data_file)):
    print("Saving file" + out_dsMPS_data_file)
    hf = h5py.File(out_dsMPS_data_file, "w")
    hf.create_dataset('data_dsMPS_x',data=data_dsMPS_x)
    hf.create_dataset('data_label_y',data=data_label_y)
    hf.close()
else: 
    print("File (" + out_dsMPS_data_file + ") already exists")




iter=0 with file=D:/Dropbox/Python/MLmodels/Datasets/mps_gp_sgram/level65_dBspl_clean_ds/Chut/ds_Chut_2_Feb_07_2022_51861688_ms_101198_101787.mat
iter=100 with file=D:/Dropbox/Python/MLmodels/Datasets/mps_gp_sgram/level65_dBspl_clean_ds/Chut/ds_Chut_2_Feb_09_2022_59003176_ms_28805_29568.mat
iter=200 with file=D:/Dropbox/Python/MLmodels/Datasets/mps_gp_sgram/level65_dBspl_clean_ds/Chut/ds_Chut_2_Mar_16_2022_50426XXX_ms_43550_44535.mat
iter=300 with file=D:/Dropbox/Python/MLmodels/Datasets/mps_gp_sgram/level65_dBspl_clean_ds/Chut/ds_Chut_3_Feb_09_2022_57194776_ms_91370_91831.mat
iter=400 with file=D:/Dropbox/Python/MLmodels/Datasets/mps_gp_sgram/level65_dBspl_clean_ds/Chut/ds_Chut_3_Feb_11_2022_60915798_ms_94915_95837.mat


KeyboardInterrupt: 